In [2]:
using Gurobi
using CSV
using DataFrames
using JuMP

┌ Info: Recompiling stale cache file C:\Users\steph\.julia\compiled\v1.0\Gurobi\do9v6.ji for Gurobi [2e9cd046-0924-5485-92f1-d5272153d98b]
└ @ Base loading.jl:1190
┌ Info: Recompiling stale cache file C:\Users\steph\.julia\compiled\v1.0\CSV\HHBkp.ji for CSV [336ed68f-0bac-5ca0-87d4-7b16caf5d00b]
└ @ Base loading.jl:1190
┌ Info: Recompiling stale cache file C:\Users\steph\.julia\compiled\v1.0\JuMP\DmXqY.ji for JuMP [4076af6c-e467-56ae-b986-b466b2749572]
└ @ Base loading.jl:1190


In [1]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

  Updating registry at `C:\Users\steph\.julia\registries\General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`


In [9]:
df = CSV.read("ev_constraints.csv", header = true, missingstring="\n")

,station_1,station_2,station_3,station_4,station_5,station_6,station_7,station_8,station_9
,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰
1,2,6,missing,missing,missing,missing,missing,missing,missing
2,6,missing,missing,missing,missing,missing,missing,missing,missing
3,3,4,5,19,missing,missing,missing,missing,missing
4,4,5,19,missing,missing,missing,missing,missing,missing
5,5,19,missing,missing,missing,missing,missing,missing,missing
6,3,4,5,19,missing,missing,missing,missing,missing
7,4,5,19,missing,missing,missing,missing,missing,missing
8,3,4,5,16,missing,missing,missing,missing,missing
9,3,4,5,16,missing,missing,missing,missing,missing


In [10]:
num_nodes = 50

m = Model(solver=GurobiSolver(OutputFlag=0))

@variable(m, x[1:num_nodes],Bin)

for row in eachrow(df)
    v = vec(convert(Array, row)) # no transpose necessary
    @constraint(m, sum(x[i] for i in v if !ismissing(i)) >= 1)
end

@objective(m, Min, sum(x[i] for i in 1:num_nodes))


┌ Warning: `convert(::Type{Array}, dfr::DataFrameRow)` is deprecated, use `permutedims(Vector(dfr))` instead.
│   caller = top-level scope at In[10]:8
└ @ Core .\In[10]:8


x[1] + x[2] + x[3] + x[4] + x[5] + x[6] + x[7] + x[8] + x[9] + x[10] + x[11] + x[12] + x[13] + x[14] + x[15] + x[16] + x[17] + x[18] + x[19] + x[20] + x[21] + x[22] + x[23] + x[24] + x[25] + x[26] + x[27] + x[28] + x[29] + x[30] + x[31] + x[32] + x[33] + x[34] + x[35] + x[36] + x[37] + x[38] + x[39] + x[40] + x[41] + x[42] + x[43] + x[44] + x[45] + x[46] + x[47] + x[48] + x[49] + x[50]

In [11]:
status = solve(m)

Academic license - for non-commercial use only


:Optimal

In [15]:
Pkg.rm("Gurobi")
Pkg.add("Gurobi")
Pkg.build("Gurobi")

  Updating `C:\Users\steph\Downloads\15.053_2019SP-master (2)\15.053_2019SP-master\Project.toml`
  [2e9cd046] - Gurobi v0.6.0
  Updating `C:\Users\steph\Downloads\15.053_2019SP-master (2)\15.053_2019SP-master\Manifest.toml`
  [2e9cd046] - Gurobi v0.6.0
 Resolving package versions...
  Updating `C:\Users\steph\Downloads\15.053_2019SP-master (2)\15.053_2019SP-master\Project.toml`
  [2e9cd046] + Gurobi v0.6.0
  Updating `C:\Users\steph\Downloads\15.053_2019SP-master (2)\15.053_2019SP-master\Manifest.toml`
  [2e9cd046] + Gurobi v0.6.0
  Building Gurobi → `C:\Users\steph\.julia\packages\Gurobi\dlJep\deps\build.log`


In [14]:
for row = 1:length(getvalue(x))
    if getvalue(x)[row] == 1.0
        println(row)
    end
end

5
6
13
16
20
31
35
39


,station_1,station_2,station_3
1,2,missing,missing


In [64]:
 ismissing(df[1,1])

false

In [68]:
m

Feasibility problem with:
 * 114 linear constraints
 * 18 variables: 18 binary
Solver is Gurobi